In [16]:
import tweepy
from tweepy.auth import OAuthHandler
import pandas as pd
import preprocessor as p

In [3]:
import json

credentials = json.load(open('credentials.json'))

consumer_key = credentials['consumer_key']
consumer_secret = credentials['consumer_secret']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [36]:
def search_tweets(searchTerm):
    n = 1000
    tweets = tweepy.Cursor(api.search_tweets,q=searchTerm,lang="en",tweet_mode="extended").items(n)
    tweets_json=[]
    for tweet in tweets:
        tweets_json.append(tweet._json)
    return tweets_json

In [38]:
# covid_tweets = search_tweets('covid')
omicron_tweets = search_tweets('omicron')
# world_economy_tweets = search_tweets('world economy')
# remittance_tweets = search_tweets('remittance')

In [32]:
def tweets_to_csv(tweets_json, name='covid'):
    tweets = [tweet['full_text'] for tweet in tweets_json]
    tweet_df = pd.DataFrame({'tweet': tweets})
    tweet_df['tweet'] = tweet_df['tweet'].apply(p.clean)
    tweet_df.to_csv(f'{name}_tweets.csv', index=False)

In [33]:
# tweets_to_csv(covid_tweets, 'covid')
tweets_to_csv(omicron_tweets, 'omicron')
# tweets_to_csv(world_economy_tweets, 'world_economy')
# tweets_to_csv(remittance_tweets, 'remittance')

In [11]:
covid_tweets['_json']

TypeError: list indices must be integers or slices, not str